<div style="text-align: center;">
    <h1 style="color: red;">Vision Transformers</h1>

    
    
</div>

## I - Introduction

L'architecture des transformers est à la base des Large Language Models(LLM) comme GPT qui nous aide désormais tous les jours pour réaliser de nombreuses tâches allant de la traduction à la génération de texte. 
Toutefois, les transformers ne sauraient rester confiner au langage. 
Le papier de  Dosovitskiy et al(2021), An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale présentent l'application de ce type de modèle pour le domaine de la vision par ordinateur, et plus précisément, la classification d'images.
Quand bien même les mots et les images ne semblent pas avoir beaucoup en commun, nous allons voir à travers le détail de la méthode des Vision transformers (ViT) que dans les deux cas leur sens caché peut être découvert et analysé.
Dans ce notebook, nous suivrons ensemble une progression linéaire commençant du pre-processing des images jusqu'à l'entrainement du modèle de transformeur. Ainsi, nous n'échapperons pas à une solide partie sur l'architecture du transformeur, en particulier l'encodeur, qui est au coeur du sujet du notebook.

## II - the ViT model

### a) Découpage en patch

Les transformeurs sont des modèles de type séquence-à-séquence, c'est à dire qu'ils prennent une séquence en entrée (souvent une phrase donc une séquence de mots) et renvoient une autre séquence en sortie. Pour pouvoir appliquer ce type d'architecture à des images, il faut tout d'abord les transformer en séquence. Pour ce faire, l'idée est de découper l'image en patch de taille carré fixe, puis de les accoler les uns à la suite des autres. Nous avons alors une image vectorisée de taille $T*PT*C$ qui n'est pas encore sous la bonne forme. Il reste donc à aplatir chacun des patchs en concaténant chacune des lignes des 3 channels pour obtenir des patchs de taille $\mathbb{R}^{T^2 \cdot C}$

Photo Patching

Par la suite, de la même manière que pour les séquence de mots, il va falloir représenter chacun des patchs obtenu par un vecteur caché encapsulant l'information contenu localement dans cette partie de l'image. c'est une étape importante car cela permet au modèle de manipuler les infos contenus dans les patchs et initialise le prochain processus d'attention qui fait la particularité des transformeurs. 
L'encapsulation, ou embedding en anglais, est dans les faits une projection sur un espace de dimension D (la dimension que l'on veut pour nos vecteurs) grâce à une couche linéaire de réseau de neurones que nous connaissons bien. L'intérêt de cette étape est multiple : 
- Le vecteur nouvellement obtenu peut maintenant être utilisé comme entrée d'un transformeur où il fera l'objet du processus d'attention.
- L'information contenu dans un objet de $\mathbb{R}^{T^2 \cdot C}$ est condensé en un vecteur de $ℝ^E$ avec E la taille des embeddings
- L'information contenu est extraite sous forme de caractéristiques pertinentes pour la représentation du patch de l'image.

$$
Z = W X + b
$$

Avec :
- W de taille  $D * T^2C$
- b de taille $\mathbb{R}^D$


L'important est que la matrice des poids et le vecteur de biais soit les même pour réaliser l'encapsulation de chacun des patchs. Nous obtenons alors une séquence de vecteurs pour l'image correspondant aux embeddings de chacun des patchs de l'image. Cela ouvre donc la possibilité d'utiliser cette entrée dans un modèle de transformeur.

Mais qu'est ce qu'un tansformeur exactement ? Nous devons nous plonger dans les méchanismes de ce modèle afin de comprendre quelles transformations nos séquences (images) vont subir et quelles informations nous allons en retirer. 


$$
L(\pi_\theta(s) \,|\, \pi_t(s)) = \sum_{i=1}^{|A|} \pi_t(a_i \,|\, s) \log \frac{\pi_t(a_i \,|\, s)}{\pi_\theta(a_i \,|\, s)}
$$
